#### Capstone Project
Arthur: Wilson Lau
Date: 2015 November

### Capstone Milestone Report

## Introduction
The problem I want to tackle in my capstone project is to detect investment opportunities for cash flow real estate property.  

## The Problem
Real estate investment is a passive income opportunity for investors to accumulate return without active management.  Huge long term appreciation is very much possible for real estate.  However, appreciation often happens in short period of time and the appreciation is heavily affected local market condition.  It is difficult for out-of-the-market investors to detect the trend and acquire properties before price goes up.  My capstone project is trying to help investors to detect real estate investment opportunities and be able to predict which zip code will experience long term appreciation in the near future. 

My project is divided into two stages:

# Stage One
- the first stage is to use anomaly detection technique to discover which zipcodes have success sprike of upward price appreciation. 
- The Anomaly detection is compared new monthly updated home price with historical price as well as comparing with the home price of the surrending areas based on neighborhoods, city, menpolian area, and state.

# Stage Two
- Based on the result of stage one, more data will be collected for the zipcodes that have sudden increase of home price.  
- Those new dataset will be used to figure out what feature contributes to the price appreciation and a predictive model will be created using those features. 

## Dateset
Dataset used is downloaded from Quandl's Zillow data.  This dataset is a monthly time series recording the change of home price. 

# What important fields and information does the data set have?
The important columns for this dataset is time(date end of the month), home price and zipcode. 



# What are its limitations i.e. what are some questions that you cannot answer with this data set?
This dataset actually have 22 columns.  For example, it has columns for different home size e.g. 1 bedroom, 2 bedroom, 3 bedroom, price per square foot, average listing price, etc...  Since it seems that most of these columns are highly corelate to home price, they may be very help for predicting home price. 

# What kind of cleaning and wrangling did you need to do?
Quandl's API is very powerful and very flexible.  At the same time, it is a bit complicated to pull the home price.  One way to pull home price is to pass zipcode as input parameter, and Quandl will return the historical price data with all the columns related to that zipcode.  So, to get the data, I will need to use a zipcode dataset that contains all the zipcode of the US and then use the Quandl API to pull data one zipcode at a time. 

# Are there other datasets you can find, use and combine with, to answer the questions that matter?
To order to group home price data by city, state, etc.., I will need to find other dataset that relates individual zipcodes with their city name and state name.  I will need to merge the dataset together in order to find out which zipcode belongs to which city or state. 

## Preliminary Findings and Hypothesis test.
.....






In [9]:
''' Problem: Try to find which zipcode and in which months has its price increase sigificantly higher than the mean price with
the same zipcode and also than the rest of the zipcode. '''

#load home price data into dataframe
import numpy as np
import pandas as pd
import pickle
from scipy import stats
import os
import glob
from os import path
import dateUtility
import datetime as dt
pd.set_option('display.max_columns', 500)

def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

#load home price data from Datafiles
# df = pickle.load( open( ".\DataPfiles\93063_to_93524.p", "rb" ) ) 
df = pd.DataFrame()
for filename in glob.glob(os.path.join('.\\DataPfiles\\', '9*.p')):
    df = pd.concat([df, pd.read_pickle(filename)], axis=0)
# there may be a problem here.  There are only about 1700 zipcode in the .p file although CA has over 2700 zipcode

### Pivot indivudal columns into a new dataframe for analysis: 
### 1) Put 1) All Homes prirce 2) Price-To-Rent Ratio into rows and grouped them by zipcode
df = df.reset_index()
dfAllHomes = df.pivot(index='index', columns='ZipCode', values='All Homes')
dfPriceToRent = df.pivot(index='index', columns='ZipCode', values='Price-to-Rent Ratio')  #I don't make use of this feature now,but will do later

### calcaluate the home price's moving average for every 3 months, with minimal of 3 months
dfAllHomesMovingAve = pd.rolling_mean(dfAllHomes,window=3,min_periods=3)

### shift entire set of Moving Average number downward for one month
### This is needed to line up the previous month moving average with the current month moving average in the same row
dfAllHomesMovingAve = dfAllHomesMovingAve.shift(periods=1,freq=None,axis=0)

### since I shift the all rows for one month earlier, so I can just subtract the two dataframe and calculate the price change.
dfAllHomesDiffFromMovAve = dfAllHomes - dfAllHomesMovingAve #price change
dfAllHomesDiffFromMovAvePercent = (dfAllHomes - dfAllHomesMovingAve)/dfAllHomes.shift(periods=1,freq=None,axis=0) #price percentage change
NumOfMonthForward = 6 #set the number of month to include for calculating the predicting average percentage increase
dfAllHomesDiffFromMovAvePercentMovAve = pd.rolling_mean(dfAllHomesDiffFromMovAvePercent,window=NumOfMonthForward,min_periods=NumOfMonthForward) # calculate the moving average percentage increase for 6 months
dfAllHomesDiffFromMovAvePercentMovAve2 = dfAllHomesDiffFromMovAvePercentMovAve.shift(periods=-(NumOfMonthForward-1),freq=None,axis=0) # shift dataframe upward for 5 record so the current month shows the moving average percentage increase for the next 6 months(including current month)
dfAllHomesStdDevInZipCode = dfAllHomesDiffFromMovAvePercentMovAve2.copy() #dataframe to hold z score within zipcode

##loop through each zipcode column
for x in dfAllHomesStdDevInZipCode.columns:
    if sum(dfAllHomesStdDevInZipCode[x].isnull()) > 0:  # if any month has null value, drop the entire zipcode
        dfAllHomesStdDevInZipCode.drop(x,axis=1)
    dfAllHomesStdDevInZipCode[x] = (dfAllHomesStdDevInZipCode[x] - dfAllHomesStdDevInZipCode[x].mean()) / dfAllHomesStdDevInZipCode[x].std()  #calculate z score for All Homes price each month

##Calculate z score comparing price across zipcode
dfAllHomesStdDevAcrossZipCode = dfAllHomesDiffFromMovAvePercent.copy() #dataframe to hold z score across different zipcode
dfAllHomesStdDevAcrossZipCode['MeanHomePrice'] = dfAllHomesStdDevAcrossZipCode.mean(axis=1)
dfAllHomesStdDevAcrossZipCode['StdDevHomePrice'] = dfAllHomesStdDevAcrossZipCode.std(axis=1)
dfAllHomesStdDevAcrossZipCode = (dfAllHomesStdDevAcrossZipCode - dfAllHomesStdDevAcrossZipCode['MeanHomePrice']) / dfAllHomesStdDevAcrossZipCode['StdDevHomePrice']

##drop the two added columns so that match the dataframe size of df..InZipCode and df..AcrossZipCode
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('MeanHomePrice',axis=1)
dfAllHomesStdDevAcrossZipCode = dfAllHomesStdDevAcrossZipCode.drop('StdDevHomePrice',axis=1)

##Abnormally Detection method ONE
##Find out which zipcode and its time frame in the dataframe has z score large than 2 in both df..InZipCode and df..AcrossZipCode dataframe
targetZscore = 2
targetZipcodeBoolean = ((dfAllHomesStdDevInZipCode > targetZscore) & (dfAllHomesStdDevAcrossZipCode > targetZscore))
# targetZipCodes = dfAllHomesMovingAve[targetZipcodeBoolean]#.dropna
selectedZipCodes = pd.DataFrame(dfAllHomesMovingAve[targetZipcodeBoolean].sum(axis = 0) > 0)  #find out which zipcodes are over target zscore
selectedZipCodes[selectedZipCodes[0] == True] #extract only the zipcode that are over zscore

### pivot a new dataframe moving targeted zipcodes into index joining the month-end time (multi-level) index
TargetZipCode = pd.DataFrame(targetZipcodeBoolean.stack())  # stack zipcode from column names into column
TargetZipCode.columns = ['PredictZipCode'] # specify columne name
TargetZipCode = pd.DataFrame(TargetZipCode.reset_index()) # move index into column
dfPredictZipCode = pd.merge(df,TargetZipCode,on=['index','ZipCode']) #merge targeted zipcode column for prediction into main dataframe(which is loaded from 9*.p files)
dfPredictZipCode.rename(columns={'index':'Month'}, inplace=True)
dfPredictZipCode['Month'] = dfPredictZipCode['Month'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
dfPredictZipCode.set_index(['Month','ZipCode'],inplace=True)
print sum(dfPredictZipCode.PredictZipCode)


### Clean irsdatafull dataframe by removing np.nan, nan(string), ' '(space) and 0 in the cells and fill those cells with back filling value
# finding dataframe cell with 'nan',' ' and 0 and then turn them into np.nan
for x in dfPredictZipCode.columns:
    if x == 'PredictZipCode':
        print "found predictZipCode"
        break
    nalist = dfPredictZipCode[x].isin(['nan',' ',0])
    dfPredictZipCode.ix[nalist,x] = np.nan

# use fillna funcion to back filling the value from following rows back to rows above
dfPredictZipCode = dfPredictZipCode.reset_index()  
dfPredictZipCode.set_index("Month")
dfPredictZipCode = dfPredictZipCode.fillna(method='bfill')
dfPredictZipCode = dfPredictZipCode.fillna(method='ffill')
dfPredictZipCode = dfPredictZipCode.reset_index()
dfPredictZipCode.set_index(['Month','ZipCode'],inplace=True)
dfPredictZipCode = dfPredictZipCode.drop('index',axis=1)
dfPredictZipCode.head()

#check to see if there is still nan like value in the dataframe
print "Predict Zip:"
print sum(dfPredictZipCode.PredictZipCode)
print "nan:"
print sum(dfPredictZipCode.isin(['nan',' ',0]).values)
print "null value:"
print sum(dfPredictZipCode.isnull().values)

### tasks to do
### process IRS dataaset - DONE
### process Census dataset - DONE
### process proximityone dataset
### apply training model to learn how IRS and Census dataset can predict target zipcodes
### evaluate and test model by breaking down into training data and test data
### apply feature scaling to dataset
### use PCA to combine some features e.g. I think I can combine all the home price for different number of bedrooms into one feature
### modify withinZipcode detection not to use z score, but just average monthly price increase over a certain percentage
### use F1 score and some other ways to evaluate the performance of the model
### apply visualization on data; add Google maps to show the results
### rewrite Storytelling





570
found predictZipCode
Predict Zip:
570
nan:
[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0 307947]
null value:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
# print "nan:"
# print sum(dfAllHomesMovingAve.isin(['nan',' ',0]).values)
# print "null value:"
# print sum(dfAllHomesMovingAve.isnull().values)
# sum(TargetZipCode.PredictZipCode)
dfPredictZipCode1 = pd.DataFrame(dfPredictZipCode.PredictZipCode)
dfPredictZipCode1.unstack(level=1)

PredictZipCode                                                   \
ZipCode             90001  90002  90003  90004  90005  90006  90007  90008   
Month                                                                        
1996-04-30          False  False  False  False  False  False  False  False   
1996-05-31          False  False  False  False  False  False  False  False   
1996-06-30          False  False  False  False  False  False  False  False   
1996-07-31          False  False  False  False  False  False  False  False   
1996-08-31          False  False  False  False  False  False  False  False   
1996-09-30          False  False  False  False  False  False  False  False   
1996-10-31          False  False  False  False  False  False  False  False   
1996-11-30          False  False  False  False  False  False  False  False   
1996-12-31          False  False  False  False  False  False  False  False   
1997-01-31          False  False  False  False  False  False  False  False   
1997-02-28          False  False  False  False  False  False  False  False   
1997-03-31          False  False  False  False  False  False  False  False   
1997-04-30          False  False  False  False  False  False  False  False   
1997-05-31          False  False  False  False  False  False  False  False   
1997-06-30          False  False  False  False  False  False  False  False   
1997-07-31          False  False  False  False  False  False  False  False   
1997-08-31          False  False  False  False  False  False  False  False   
1997-09-30          False  False  False  False  False  False  False  False   
1997-10-31          False  False  False  False  False  False  False  False   
1997-11-30          False  False  False  False  False  False  False  False   
1997-12-31          False  False  False  False  False  False  False  False   
1998-01-31          False  False  False  False  False  False  False  False   
1998-02-28          False  False  False  False  False  False  False  False   
1998-03-31          False  False  False  False  False  False  False  False   
1998-04-30          False  False  False  False  False  False  False  False   
1998-05-31          False  False  False  False  False  False  False  False   
1998-06-30          False  False  False  False  False  False  False  False   
1998-07-31          False  False  False  False  False  False  False  False   
1998-08-31          False  False  False  False  False  False  False  False   
1998-09-30          False  False  False  False  False  False  False  False   
...                   ...    ...    ...    ...    ...    ...    ...    ...   
2013-05-31          False  False  False  False  False  False  False  False   
2013-06-30          False  False  False  False  False  False  False  False   
2013-07-31          False  False  False  False  False  False  False  False   
2013-08-31          False  False  False  False  False  False  False  False   
2013-09-30          False  False  False  False  False  False  False  False   
2013-10-31          False  False  False  False  False  False  False  False   
2013-11-30          False  False  False  False  False  False  False  False   
2013-12-31          False  False  False  False  False  False  False  False   
2014-01-31          False  False  False  False  False  False  False  False   
2014-02-28          False  False  False  False  False  False  False  False   
2014-03-31          False  False  False  False  False  False  False  False   
2014-04-30          False  False  False  False  False  False  False  False   
2014-05-31          False  False  False  False  False  False  False  False   
2014-06-30          False  False  False  False  False  False  False  False   
2014-07-31          False  False  False  False  False  False  False  False   
2014-08-31          False  False  False  False  False  False  False  False   
2014-09-30          False  False  False  False  False  False  False  False   
2014-10-31          False  False 

In [11]:
dfPredictZipCodetest1 = dfPredictZipCode.reset_index()
dfPredictZipCodetest1[dfPredictZipCodetest1.ZipCode == '96097']

,Month,ZipCode,All Homes,1 Bedroom,Median List Price per Square Foot,2 Bedroom,Number of Homes for Rent,4 Bedroom,5 or More Bedroom,Ratio of Sale Price to List Price,"Estimated Rent, All Homes in Region",Median List Price,Median Sale Price per Square Foot,Price per Square Foot,Median Sale Price,Single Family Residences,Listings with Price Cut in Last 30 Days,Condominiums,"Median Rent, Homes Listed for Rent",3 Bedroom,Estimated Rent per Square Foot,Percentage of Sales that were Foreclosures,Price-to-Rent Ratio,PredictZipCode
303941,1996-04-30,96097,76900,139800,166.111574,66000,1.8581,120000,816400,0.9842,1281,259000,66.894531,60,96000.0000,77300,5.6338,760000,325,80400,0.974,9.0909,9.41,False
303942,1996-05-31,96097,77300,139800,166.111574,65400,1.8581,125000,816400,0.9842,1281,259000,66.894531,61,96000.0000,77900,5.6338,760000,325,81700,0.974,9.0909,9.41,False
303943,1996-06-30,96097,77700,139800,166.111574,64500,1.8581,127600,816400,0.9842,1281,259000,66.894531,60,96000.0000,78300,5.6338,760000,325,82700,0.974,9.0909,9.41,False
303944,1996-07-31,96097,77900,139800,166.111574,64200,1.8581,126200,816400,0.9842,1281,259000,66.894531,60,96000.0000,78500,5.6338,760000,325,83000,0.974,9.0909,9.41,False
303945,1996-08-31,96097,78800,139800,166.111574,64500,1.8581,125600,816400,0.9842,1281,259000,66.894531,60,96000.0000,79300,5.6338,760000,325,83100,0.974,9.0909,9.41,False
303946,1996-09-30,96097,79700,139800,166.111574,65000,1.8581,125700,816400,0.9842,1281,259000,64.537000,61,83307.6923,80100,5.6338,760000,325,83100,0.974,9.0909,9.41,False
303947,1996-10-31,96097,80200,139800,166.111574,65400,1.8581,125500,816400,0.9842,1281,259000,64.537000,61,83307.6923,80800,5.6338,760000,325,83800,0.974,9.0909,9.41,False
303948,1996-11-30,96097,81200,139800,166.111574,65800,1.8581,124400,816400,0.9842,1281,259000,61.633282,61,84500.0000,82100,5.6338,760000,325,85200,0.974,9.0909,9.41,False
303949,1996-12-31,96097,82300,139800,166.111574,66000,1.8581,123000,816400,0.9842,1281,259000,61.633282,61,84500.0000,83100,5.6338,760000,325,86600,0.974,9.0909,9.41,False
303950,1997-01-31,96097,82500,139800,166.111574,65600,1.8581,122600,816400,0.9842,1281,259000,61.633282,62,84500.0000,83300,5.6338,760000,325,87400,0.974,9.0909,9.41,False


In [12]:
# sum(dfPredictZipCodeFinalnotstack.PredictZipCode.isnull().values)


In [13]:
###########process IRS dataset
### prepare 1998 IRS data
irsdata = pd.read_csv('./DataPfiles/98zp05ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
# irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
#           'TaxableInterestRtn','TaxableInterestAmt','EarnedIncomeCreditRtn','EarnedIncomeCreditAmt', \
#           'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalScheduleC','ScheduleFTotalRtn','ScheduleFTotalScheduleF',\
#           'ScheduleARtn','ScheduleAAmt'
#           ]
irsdata.columns= irsCol #assign column names to dataframe
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns: #clean up data by removing non numeric characters
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata1998 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata1998['AverageAGI'] = irsdata1998.AGI.astype(int) #/ irsdata1998['NumberOfReturns'])
irsdata1998['NumberOfReturns'] = irsdata1998['NumberOfReturns'].astype(int)
irsdata1998['AverageAGI'] = irsdata1998['AverageAGI'] * 1000 / irsdata1998.NumberOfReturns

irsdata1998full = pd.DataFrame(columns=irsdata1998.columns.to_series().append(pd.Series("Month")))
# irsdata1998full = irsdata1998.copy()

for x in range(1,irsdata1998.shape[0]):
    listtemp = []
    irsdata1998t = pd.DataFrame(columns=irsdata1998.columns)
    irsdata1998t = irsdata1998t.append([irsdata1998.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("1998-%s-02"%thisMonth)
#         print dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d")
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata1998t["Month"] = pd.Series(listtemp)
    irsdata1998full = irsdata1998full.append(irsdata1998t, ignore_index=True)

irsdata1998full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe
irsdata1998small = irsdata1998full[irsdata1998full.index.get_level_values('Month') == '1998-03-31']
irsdata1998small2 = irsdata1998full[irsdata1998full.index.get_level_values('ZipCode') == '90011']

# irsdata1998small = irsdata1998full[irsdata1998full.index.get_level_values('Month') == '1998-03-31']
### merge IRS data with zipcode price data
irsdatafull = irsdata1998full
print "done"
# dfPredictZipCodesmall = dfPredictZipCode[dfPredictZipCode.index.get_level_values('Month') == '1998-03-31']
# dfPredictZipCodesmall = pd.concat([dfPredictZipCodesmall.unstack(level=1),irsdata1998small.unstack(level=1)], axis=1)
# dfPredictZipCodesmall.stack(level=1)

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

2396
done


In [14]:
### prepare 2001 IRS data
irsdata = pd.read_csv('./DataPfiles/01zp05ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2001 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2001['AverageAGI'] = irsdata2001.AGI.astype(int) #/ irsdata1998['NumberOfReturns'])
irsdata2001['NumberOfReturns'] = irsdata2001['NumberOfReturns'].astype(int)
irsdata2001['AverageAGI'] = irsdata2001['AverageAGI'] * 1000 / irsdata2001.NumberOfReturns
print irsdata2001['AverageAGI'].mean()

irsdata2001full = pd.DataFrame(columns=irsdata2001.columns.to_series().append(pd.Series("Month")))
# irsdata1998full = irsdata1998.copy()

for x in range(1,irsdata2001.shape[0]):
    listtemp = []
    irsdata2001t = pd.DataFrame(columns=irsdata2001.columns)
    irsdata2001t = irsdata2001t.append([irsdata2001.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2001-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2001t["Month"] = pd.Series(listtemp)
    irsdata2001full = irsdata2001full.append(irsdata2001t, ignore_index=True)
    
irsdata2001full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2001full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdata1998full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCodeFinal.unstack(level=1),irsdata2001full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

print "done"
# b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
# b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
# dfPredictZipCodeFinal[b1]  #for testing the result

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

2412
47996.9858028
done


In [15]:
### prepare 2002 IRS data
irsdata = pd.read_csv('./DataPfiles/zptab02ca.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
# print irsdata.columns
# print calzipcode.columns
# print irsdata.ZipCode
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2002 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2002['AverageAGI'] = irsdata2002.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2002['NumberOfReturns'] = irsdata2002['NumberOfReturns'].astype(int)
irsdata2002['AverageAGI'] = irsdata2002['AverageAGI'] * 1000 / irsdata2002.NumberOfReturns
print irsdata2002['AverageAGI'].mean()

irsdata2002full = pd.DataFrame(columns=irsdata2002.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2002.shape[0]):
    listtemp = []
    irsdata2002t = pd.DataFrame(columns=irsdata2002.columns)
    irsdata2002t = irsdata2002t.append([irsdata2002.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2002-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2002t["Month"] = pd.Series(listtemp)
    irsdata2002full = irsdata2002full.append(irsdata2002t, ignore_index=True)
    
irsdata2002full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2002full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1) #not concat until all irsdatafull is created
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

2414
49689.0262676
done


In [16]:
### prepare 2004 IRS data
irsdata = pd.read_csv('./DataPfiles/ZIPCode2004CA.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']
irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2004 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2004['AverageAGI'] = irsdata2004.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2004['NumberOfReturns'] = irsdata2004['NumberOfReturns'].astype(int)
irsdata2004['AverageAGI'] = irsdata2004['AverageAGI'] * 1000 / irsdata2004.NumberOfReturns
print irsdata2004['AverageAGI'].mean()

irsdata2004full = pd.DataFrame(columns=irsdata2004.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2004.shape[0]):
    listtemp = []
    irsdata2004t = pd.DataFrame(columns=irsdata2004.columns)
    irsdata2004t = irsdata2004t.append([irsdata2004.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2004-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2004t["Month"] = pd.Series(listtemp)
    irsdata2004full = irsdata2004full.append(irsdata2004t, ignore_index=True)
    
irsdata2004full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2004full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)  #not concat until all irsdatafull is created
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2390
52308.0969728
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
### prepare 2005 IRS data
irsdata = pd.read_csv('./DataPfiles/ZIPCode2005CA.csv',header=4, na_values=['**','--'])
irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2005 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]
irsdata2005['AverageAGI'] = irsdata2005.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2005['NumberOfReturns'] = irsdata2005['NumberOfReturns'].astype(int)
irsdata2005['AverageAGI'] = irsdata2005['AverageAGI'] * 1000 / irsdata2005.NumberOfReturns
print irsdata2005['AverageAGI'].mean()

irsdata2005full = pd.DataFrame(columns=irsdata2005.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2005.shape[0]):
    listtemp = []
    irsdata2005t = pd.DataFrame(columns=irsdata2005.columns)
    irsdata2005t = irsdata2005t.append([irsdata2005.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2005-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2005t["Month"] = pd.Series(listtemp)
    irsdata2005full = irsdata2005full.append(irsdata2005t, ignore_index=True)
    
irsdata2005full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2005full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)  #not concat until all irsdatafull is created
print "done"

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2396
55154.0374922
done


C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
### prepare 2006 IRS data
irsdata = pd.read_csv('./DataPfiles/ZIPCode2006CA.csv',header=4, na_values=['**','--'])
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2006 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2006 = irsdata2006[(irsdata2006['Range'] == irsdata2006['ZipCode'])]
irsdata2006 = irsdata2006.drop('Range', axis=1)
irsdata2006['AverageAGI'] = irsdata2006.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2006['NumberOfReturns'] = irsdata2006['NumberOfReturns'].astype(int)
irsdata2006['AverageAGI'] = irsdata2006['AverageAGI'] * 1000 / irsdata2006.NumberOfReturns
print irsdata2006['AverageAGI'].mean()

irsdata2006full = pd.DataFrame(columns=irsdata2006.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2006.shape[0]):
    listtemp = []
    irsdata2006t = pd.DataFrame(columns=irsdata2006.columns)
    irsdata2006t = irsdata2006t.append([irsdata2006.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2006-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2006t["Month"] = pd.Series(listtemp)
    irsdata2006full = irsdata2006full.append(irsdata2006t, ignore_index=True)
    
irsdata2006full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2006full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)  #not concat until all irsdatafull is created
print "done"

19128
55230.5241484
done


In [19]:
### prepare 2007 IRS data
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

irsdata = pd.read_csv('./DataPfiles/ZIPCode2007CA.csv',header=4, na_values=['**','--'])
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2007 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2007 = irsdata2007[pd.isnull(irsdata2007['Range'])]
irsdata2007 = irsdata2007.drop('Range', axis=1)  #it seems that the inplace operation has bug that failed this line, so I removed it. 
irsdata2007['AverageAGI'] = irsdata2007.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
irsdata2007['NumberOfReturns'] = irsdata2007['NumberOfReturns'].astype(int)
irsdata2007['AverageAGI'] = irsdata2007['AverageAGI'] * 1000 / irsdata2007.NumberOfReturns
print irsdata2007['AverageAGI'].mean()

irsdata2007full = pd.DataFrame(columns=irsdata2007.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2007.shape[0]):
    listtemp = []
    irsdata2007t = pd.DataFrame(columns=irsdata2007.columns)
    irsdata2007t = irsdata2007t.append([irsdata2007.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2007-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2007t["Month"] = pd.Series(listtemp)
    irsdata2007full = irsdata2007full.append(irsdata2007t, ignore_index=True)
    
irsdata2007full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2007full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)   #not concat until all irsdatafull is created

19176
52460.2197889


In [20]:
### prepare 2008 IRS data
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

irsdata = pd.read_csv('./DataPfiles/08zp05ca.csv',header=4, na_values=['**','--'])
# irsdata = pd.read_csv('./DataPfiles/08zp05ca.csv',header=4, na_values=['**','--'], \
#                       dtype={'Range': np.str, 'ZipCode': np.str, 'NumberOfReturns': np.str, \
#                             'ExemptionsRtn':np.str, 'DepedentExemptionsAmt':np.str,'AGI':np.str,'SalariesWagesRtn':np.str, \
#                             'SalariesWagesAmt':np.str,'TaxableInterestRtn':np.str, 'TaxableInterestAmt':np.str, \
#                             'TotalTaxRtn':np.str, 'TotalTaxAmt':np.str, 'ScheduleCTotalRtn': np.str,'ScheduleCTotalAmt':np.str, \
#                             'ScheduleFTotalRtn':np.str, 'ScheduleFTotalAmt':np.str, 'ScheduleARtn':np.str,'ScheduleAAmt':np.str}
#                       ) 
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol

for x in irsdata.columns:
    irsdata[x] = irsdata[x].astype(str)  #convert dtype from float64 to string object
    irsdata[x] = irsdata[x].apply(trim_fraction) #remove the decimals from the string object 
    
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2008 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2008 = irsdata2008[irsdata2008['Range'] == 'nan']
irsdata2008 = irsdata2008.drop('Range', axis=1)  #it seems that the inplace operation has bug that failed this line, so I removed it. 
irsdata2008['AverageAGI'] = irsdata2008.AGI.astype(int) #/ irsdata2008['NumberOfReturns'])
irsdata2008['NumberOfReturns'] = irsdata2008['NumberOfReturns'].astype(int)
irsdata2008['AverageAGI'] = irsdata2008['AverageAGI'] * 1000 / irsdata2008.NumberOfReturns
print irsdata2008['AverageAGI'].mean()

irsdata2008full = pd.DataFrame(columns=irsdata2008.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2008.shape[0]):
    listtemp = []
    irsdata2008t = pd.DataFrame(columns=irsdata2008.columns)
    irsdata2008t = irsdata2008t.append([irsdata2008.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2008-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2008t["Month"] = pd.Series(listtemp)
    irsdata2008full = irsdata2008full.append(irsdata2008t, ignore_index=True)
    
irsdata2008full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2008full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1) #not concat until all irsdatafull is created
print "done"

15600
56658.0840399
done


In [21]:
### prepare 2009 IRS data
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

irsdata = pd.read_csv('./DataPfiles/09zp05ca.csv',header=4, na_values=['**','--'])
# irsdata = pd.read_csv('./DataPfiles/08zp05ca.csv',header=4, na_values=['**','--'], \
#                       dtype={'Range': np.str, 'ZipCode': np.str, 'NumberOfReturns': np.str, \
#                             'ExemptionsRtn':np.str, 'DepedentExemptionsAmt':np.str,'AGI':np.str,'SalariesWagesRtn':np.str, \
#                             'SalariesWagesAmt':np.str,'TaxableInterestRtn':np.str, 'TaxableInterestAmt':np.str, \
#                             'TotalTaxRtn':np.str, 'TotalTaxAmt':np.str, 'ScheduleCTotalRtn': np.str,'ScheduleCTotalAmt':np.str, \
#                             'ScheduleFTotalRtn':np.str, 'ScheduleFTotalAmt':np.str, 'ScheduleARtn':np.str,'ScheduleAAmt':np.str}
#                       ) 
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol

for x in irsdata.columns:
    irsdata[x] = irsdata[x].astype(str)  #convert dtype from float64 to string object
    irsdata[x] = irsdata[x].apply(trim_fraction) #remove the decimals from the string object 
    
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2009 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2009 = irsdata2009[irsdata2009['Range'] == 'nan']
irsdata2009 = irsdata2009.drop('Range', axis=1)  #it seems that the inplace operation has bug that failed this line, so I removed it. 
irsdata2009['AverageAGI'] = irsdata2009.AGI.astype(int) #/ irsdata2009['NumberOfReturns'])
irsdata2009['NumberOfReturns'] = irsdata2009['NumberOfReturns'].astype(int)
irsdata2009['AverageAGI'] = irsdata2009['AverageAGI'] * 1000 / irsdata2009.NumberOfReturns
print irsdata2009['AverageAGI'].mean()

irsdata2009full = pd.DataFrame(columns=irsdata2009.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2009.shape[0]):
    listtemp = []
    irsdata2009t = pd.DataFrame(columns=irsdata2009.columns)
    irsdata2009t = irsdata2009t.append([irsdata2009.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2009-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2009t["Month"] = pd.Series(listtemp)
    irsdata2009full = irsdata2009full.append(irsdata2009t, ignore_index=True)
    
irsdata2009full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2009full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1) #not concat until all irsdatafull is created
print "done"

10409
55546.2507987
done


In [22]:
### prepare 2010 IRS data
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

irsdata = pd.read_csv('./DataPfiles/10zp05ca.csv',header=4, na_values=['**','--'])
# irsdata = pd.read_csv('./DataPfiles/08zp05ca.csv',header=4, na_values=['**','--'], \
#                       dtype={'Range': np.str, 'ZipCode': np.str, 'NumberOfReturns': np.str, \
#                             'ExemptionsRtn':np.str, 'DepedentExemptionsAmt':np.str,'AGI':np.str,'SalariesWagesRtn':np.str, \
#                             'SalariesWagesAmt':np.str,'TaxableInterestRtn':np.str, 'TaxableInterestAmt':np.str, \
#                             'TotalTaxRtn':np.str, 'TotalTaxAmt':np.str, 'ScheduleCTotalRtn': np.str,'ScheduleCTotalAmt':np.str, \
#                             'ScheduleFTotalRtn':np.str, 'ScheduleFTotalAmt':np.str, 'ScheduleARtn':np.str,'ScheduleAAmt':np.str}
#                       ) 
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol

for x in irsdata.columns:
    irsdata[x] = irsdata[x].astype(str)  #convert dtype from float64 to string object
    irsdata[x] = irsdata[x].apply(trim_fraction) #remove the decimals from the string object 
    
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2010 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2010 = irsdata2010[irsdata2010['Range'] == 'nan']
irsdata2010 = irsdata2010.drop('Range', axis=1)  #it seems that the inplace operation has bug that failed this line, so I removed it. 
irsdata2010['AverageAGI'] = irsdata2010.AGI.astype(int) #/ irsdata2010['NumberOfReturns'])
irsdata2010['NumberOfReturns'] = irsdata2010['NumberOfReturns'].astype(int)
irsdata2010['AverageAGI'] = irsdata2010['AverageAGI'] * 1000 / irsdata2010.NumberOfReturns
print irsdata2010['AverageAGI'].mean()

irsdata2010full = pd.DataFrame(columns=irsdata2010.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2010.shape[0]):
    listtemp = []
    irsdata2010t = pd.DataFrame(columns=irsdata2010.columns)
    irsdata2010t = irsdata2010t.append([irsdata2010.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2010-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2010t["Month"] = pd.Series(listtemp)
    irsdata2010full = irsdata2010full.append(irsdata2010t, ignore_index=True)
    
irsdata2010full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge IRS data with zipcode price data
irsdatafull = irsdatafull.append(irsdata2010full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1) #not concat until all irsdatafull is created
print "done"

10402
56226.670196
done


In [23]:
### prepare 2011 IRS data
def trim_fraction(text):
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

irsdata = pd.read_csv('./DataPfiles/11zp05ca.csv',header=4, na_values=['**','--'])
# irsdata = pd.read_csv('./DataPfiles/08zp05ca.csv',header=4, na_values=['**','--'], \
#                       dtype={'Range': np.str, 'ZipCode': np.str, 'NumberOfReturns': np.str, \
#                             'ExemptionsRtn':np.str, 'DepedentExemptionsAmt':np.str,'AGI':np.str,'SalariesWagesRtn':np.str, \
#                             'SalariesWagesAmt':np.str,'TaxableInterestRtn':np.str, 'TaxableInterestAmt':np.str, \
#                             'TotalTaxRtn':np.str, 'TotalTaxAmt':np.str, 'ScheduleCTotalRtn': np.str,'ScheduleCTotalAmt':np.str, \
#                             'ScheduleFTotalRtn':np.str, 'ScheduleFTotalAmt':np.str, 'ScheduleARtn':np.str,'ScheduleAAmt':np.str}
#                       ) 
irsCol = ['Range', 'ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
          'TaxableInterestRtn','TaxableInterestAmt', \
          'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleCTotalAmt','ScheduleFTotalRtn','ScheduleFTotalAmt',\
          'ScheduleARtn','ScheduleAAmt'
          ]
irsdata.columns= irsCol

for x in irsdata.columns:
    irsdata[x] = irsdata[x].astype(str)  #convert dtype from float64 to string object
    irsdata[x] = irsdata[x].apply(trim_fraction) #remove the decimals from the string object 
    
irsdata['ZipCode'] = irsdata['ZipCode'].astype(str)  #convert dtype from float64 to string object
irsdata['ZipCode'] = irsdata['ZipCode'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleCTotalAmt'] = irsdata['ScheduleCTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleFTotalAmt'] = irsdata['ScheduleFTotalAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['ScheduleAAmt'] = irsdata['ScheduleAAmt'].apply(trim_fraction) #remove the decimals from the string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].astype(str)  #convert dtype from float64 to string object
irsdata['DepedentExemptionsAmt'] = irsdata['DepedentExemptionsAmt'].apply(trim_fraction) #remove the decimals from the string object
chars_to_remove = ['*', '**', '--',',']
for col in irsdata.columns:
    irsdata[col] = irsdata[col].str.translate(None, ''.join(chars_to_remove))

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
irsdata2011 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

irsdata2011 = irsdata2011[irsdata2011['Range'] == 'nan']
irsdata2011 = irsdata2011.drop('Range', axis=1)  #it seems that the inplace operation has bug that failed this line, so I removed it. 
irsdata2011['AverageAGI'] = irsdata2011.AGI.astype(int) #/ irsdata2011['NumberOfReturns'])
irsdata2011['NumberOfReturns'] = irsdata2011['NumberOfReturns'].astype(int)
irsdata2011['AverageAGI'] = irsdata2011['AverageAGI'] * 1000 / irsdata2011.NumberOfReturns
print irsdata2011['AverageAGI'].mean()

irsdata2011full = pd.DataFrame(columns=irsdata2011.columns.to_series().append(pd.Series("Month")))

for x in range(1,irsdata2011.shape[0]):
    listtemp = []
    irsdata2011t = pd.DataFrame(columns=irsdata2011.columns)
    irsdata2011t = irsdata2011t.append([irsdata2011.iloc[x]]*12,ignore_index=True)
    for i in range(12):
        thisMonth = ("0%i"%(i+1,))[-2:]
        d = dateUtility.mkDateTime("2011-%s-02"%thisMonth)
        listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
    irsdata2011t["Month"] = pd.Series(listtemp)
    irsdata2011full = irsdata2011full.append(irsdata2011t, ignore_index=True)
    
irsdata2011full.set_index(['Month','ZipCode'],inplace=True)  #set dataframe index to match with zipcode price dataframe

### merge zipcode price data
irsdatafull = irsdatafull.append(irsdata2011full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# print "done"

10409
55833.1618228


In [24]:
### Clean irsdatafull dataframe by removing nan(string), ' '(space) and 0 in the cells and fill those cells with back filling value

# finding dataframe cell with 'nan',' ' and 0 and then turn them into np.nan
for x in irsdatafull.columns:
    nalist = irsdatafull[x].isin(['nan',' ',0])
    irsdatafull.ix[nalist,x] = np.nan

#use fillna funcion to back filling the value from following rows back to rows above
irsdatafull = irsdatafull.reset_index() # need to reset index because the current first level index is grouped by zipcode 
irsdatafull = irsdatafull.set_index('Month') #reset index to month; rows for the same zipcode from different month are listed consecutively
irsdatafull = irsdatafull.fillna(method='bfill') #apply back filling with fillna
irsdatafull = irsdatafull.fillna(method='ffill') #apply forward filling to correct cells the last cells that didn't get fill with back filling
irsdatafull = irsdatafull.reset_index()
irsdatafull.set_index(['Month','ZipCode'],inplace=True) #putting back index of Monthly and ZipCode for merging with dfPredictZipCodeFinal later

#check to see if there is still nan like value in the dataframe
print sum(irsdatafull.isin(['nan',' ',0]).values)
print sum(irsdatafull.isnull().values)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [133]:
### merge IRS data with zipcode price data
# irsdatafull = irsdatafull.append(irsdata2011full)
dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinalnotstack = dfPredictZipCodeFinal.copy()
dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# dfPredictZipCodeFinalstack = dfPredictZipCodeFinal.copy()

In [26]:
print sum(dfPredictZipCode.isnull().values)
print sum(dfPredictZipCode.unstack(level=1).isnull().values)
tstack1 = dfPredictZipCode.unstack(level=1)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[  4   4   4 ..., 232   0 231]


In [134]:
### Clean irsdatafull dataframe by removing nan(string), ' '(space) and 0 in the cells and fill those cells with back filling value

# finding dataframe cell with 'nan',' ' and 0 and then turn them into np.nan
for x in dfPredictZipCodeFinal.columns:
    if x == 'PredictZipCode':
        print "found predictZipCode"
        break    
    nalist = dfPredictZipCodeFinal[x].isin(['nan',' ',0])
    dfPredictZipCodeFinal.ix[nalist,x] = np.nan

#use fillna funcion to back filling the value from following rows back to rows above
dfPredictZipCodeFinal['PredictZipCode'].fillna(value=False,inplace=True) # Change the NaN values created during the dataframe merge because of stack/unstack
dfPredictZipCodeFinal = dfPredictZipCodeFinal.reset_index() # need to reset index because the current first level index is grouped by zipcode 
dfPredictZipCodeFinal = dfPredictZipCodeFinal.set_index('level_0') #reset index to month; rows for the same zipcode from different month are listed consecutively
dfPredictZipCodeFinal = dfPredictZipCodeFinal.fillna(method='bfill') #apply back filling with fillna
dfPredictZipCodeFinal = dfPredictZipCodeFinal.fillna(method='ffill') #apply forward filling to correct cells the last cells that didn't get fill with back filling
dfPredictZipCodeFinal = dfPredictZipCodeFinal.reset_index()
dfPredictZipCodeFinal.rename(columns={'level_0':'Month'}, inplace=True)
#### I am disabling this line for now as making Month and ZipCode doesn't seem to add much value 
# dfPredictZipCodeFinal.set_index(['Month','ZipCode'],inplace=True) #putting back index of Monthly and ZipCode for merging with dfPredictZipCodeFinal later


#check to see if there is still nan like value in the dataframe
print "nan:"
print sum(dfPredictZipCodeFinal.isin(['nan',' ',0]).values)
print "null value:"
print sum(dfPredictZipCodeFinal.isnull().values)

found predictZipCode
nan:
[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0 414895      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0]
null value:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [32]:
# pd.set_option('display.max_rows', 500)
# print sum(dfPredictZipCodeFinal.PredictZipCode)
# dfPredictZipCodeFinal.PredictZipCode.isnull

# print sum(dfPredictZipCodeFinal['PredictZipCode'].isnull().values)
# dfPredictZipCodeFinalnotstack['PredictZipCode'].fillna(value='test', inplace=True)
# # dfp1["preTestScore"].fillna(value=False , inplace=True)
# print sum(dfPredictZipCodeFinalnotstack['PredictZipCode'].isnull().values)
dfPredictZipCodeFinal.describe()

### simpe groupig analysis ###

## Find out the average value for columns such as all home price, average AGI, etc.. grouped by targeted zipcode and non targeted
print dfPredictZipCodeFinal.groupby('PredictZipCode').mean()


selectedZipCodes[selectedZipCodes[0] == True]

                    1 Bedroom      2 Bedroom      3 Bedroom      4 Bedroom  \
PredictZipCode                                                               
False           184372.965449  279748.970944  381575.171308  509294.021861   
True            204144.736842  289568.947368  403477.894737  634006.491228   

                5 or More Bedroom      All Homes    AverageAGI   Condominiums  \
PredictZipCode                                                                  
False               764082.137890  384631.589920  50547.874187  275363.406886   
True                958855.438596  404595.964912  35138.126615  305274.736842   

                Estimated Rent per Square Foot  \
PredictZipCode                                   
False                                 1.267280   
True                                  1.370189   

                Estimated Rent, All Homes in Region  \
PredictZipCode                                        
False                                   1930.173458

,0
ZipCode,
90010,True
90013,True
90014,True
90015,True
90017,True
90028,True
90029,True
90049,True
90077,True


In [132]:
### simpe groupig analysis ###

## the number of targeted zipcode in different year.  

# dfPredictZipCodeFinal2 = dfPredictZipCodeFinal.copy()
dfPredictZipCodeFinal.reset_index(inplace = True)
dfPredictZipCodeFinal['Year'] = dfPredictZipCodeFinal.Month.str[:4]  # add a new column to store which year(first 4 digits of date) based on Month 
TargetZipCodeCount = dfPredictZipCodeFinal.groupby(['Year','PredictZipCode']).count()['ZipCode']
TargetZipCodeCount.reset_index(1)
TargetZipCodeCount = TargetZipCodeCount.rename(columns = {'ZipCode':'ZipCodeCount'})
TargetZipCodeCount

# a1 = pd.DataFrame(dfPredictZipCodeFinal.ix[dfPredictZipCodeFinal['PredictZipCode'] == True,'PredictZipCode'])
# a1.reset_index(inplace = True)
# a1.set_index('ZipCode',inplace = True)

# s = pd.Series()
# for i,t in a1['Month']:
#     print t[:4]
    
# def func(group):
#     print group.dtypes
#     x1 = group['PredictZipCode'].sum()
#     return pd.Series([x1], index=['Year'])

# a1.groupby('Month').apply(func)

# pd.Series([a1.Month.str[:4]],index=['Year'])
# a1['Year'] = a1.Month.str[:4]
# a1['Year'] = a1.index.str[:4]

# df1['timestamp'].datetime.year

#     for i in range(12):
#         thisMonth = ("0%i"%(i+1,))[-2:]
#         d = dateUtility.mkDateTime("2011-%s-02"%thisMonth)
#         listtemp.append(dateUtility.mkLastOfMonth(d).strftime("%Y-%m-%d"))
#     irsdata2011t["Month"] = pd.Series(listtemp)

# a1.reset_index().groupby('Month').count()

# a1 = a1.reset_index().groupby('Month').count()
# a1.groupby('Month').count

ValueError: cannot insert level_0, already exists

In [13]:
### Cleaning Nan data in the dataframe
dffinaltest = dfPredictZipCodeFinal.copy()

######################
##new code starts here
dffinaltest = dffinaltest.fillna(method='bfill')



##new code ends here
######################


nalist = dffinaltest['DepedentExemptionsAmt'].str.contains("NaN", na=True)
dffinaltest.ix[nalist,'DepedentExemptionsAmt'] == np.nan

for x in dffinaltest.columns:
    nalist = dffinaltest[x].str.contains("NaN", na=True)
    dffinaltest.ix[nalist,x] == np.nan
# for x in dffinaltest.columns:
#     dffinaltest[x] = dffinaltest[x].astype(int).astype(float)  
# dffinaltest1 = dffinaltest.dropna(axis=0,how='any')
dffinaltest2 = dffinaltest.fillna(dffinaltest.mean(numeric_only=True))
# dffinaltest2['SalariesWagesAmt'] = dffinaltest2['SalariesWagesAmt'].astype(float)
# dffinaltest2['AGI'] = dffinaltest2['AGI'].astype(float)
print dffinaltest2['SalariesWagesAmt'].shape
# dffinaltest2['SalariesWagesAmt'] = dffinaltest2['SalariesWagesAmt'].fillna(dffinaltest2['SalariesWagesAmt'].mean())
# dffinaltest2['SalariesWagesAmt']dffinaltest2['SalariesWagesAmt'].isnull())
dffinaltest2.info()
# dffinaltest2.describe()
# dffinaltest2['SalariesWagesAmt'] = dffinaltest2['SalariesWagesAmt'].fillna(dffinaltest2['SalariesWagesAmt'].mean())
dffinaltest2['SalariesWagesAmt']
# dffinaltest2.head(5)
# sum(dffinaltest2['AGI'].isnull())

# dffinaltest2['AverageAGI'].isnull()
# dffinaltest2.describe().ix[:,15:16]
# dffinaltest2['SalariesWagesAmt']

# # dffinaltest.mean
# dffinaltest1 = dffinaltest.dropna()
# dffinaltest1.info()
# dffinaltest2 = dffinaltest1.mean(axis=0)
# dffinaltest2
# # dffinaltest1 = dffinaltest1.ix[:,0:2]
# # dffinaltest1.mean(axis=0)
# dffinaltest2.info()
# dffinaltest2.isnan.values.any()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
# dffinaltest1 = dffinaltest.swaplevel(1,0)
# dffinaltest1 = dffinaltest1.fillna(method='bfill

dffinaltest1 = dffinaltest.copy()
dffinaltest1.index.names = ['Date','ZipCode']
# dffinaltest1 = dffinaltest.swaplevel(-1,-2)
# homeindex = pd.MultiIndex.from_tuples(tuples, names=['date', 'ZipCode'])
dffinaltest1.head()


In [ ]:
dffinaltest

In [ ]:
for x in dffinaltest2.columns:
    if dffinaltest[x].astype(str) == "Nan":
        print "yes"

In [ ]:
### This cell is trying to combine the individual california zipcode data .p pickle file into a singe file
### so the main analytic code can load all zipcodes and their distance difference into a single data frame.

import os
import glob
import pandas as pd
from os import path
path = '.\\DataPfiles\\'

calzipcodeAllLong = pd.DataFrame()    
for filename in glob.glob(os.path.join(path, 'calzip*.p')):
    calzipcodeAllLong = pd.concat([calzipcodeAllLong, pd.read_pickle(filename)], axis=1)

dfname = './DataPfiles/' + 'calzipcodeAllLong' + ".p"
pickle.dump( calzipcodeAllLong, open( dfname, "wb" ) )
print 'Done: ' + dfname

In [ ]:
##Abnormally Detection method TWO
##Find out zipcode in df..InZipCode which has z score > 2 and in df..AcrossZipCode among the 10 closest zipcodes which has z score > 2



### This cell is trying to calculate the distance between two zipcodes (within California) in miles
import pandas as pd
import pickle
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from math import radians, cos, sin, asin, sqrt
def calculateDistance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km


geolocator = Nominatim()

zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
calzipcode = zipcode[zipcode.State == 'California']

startzipcodeIndex = int(100)    #last batch processed by script at test.ipynb
endzipcodeIndex = int(200)

for z in calzipcode.iloc[startzipcodeIndex:endzipcodeIndex,0]:
# for z in calzipcode.iloc[2:3,0]:
# for z in calzipcode['Postal Code']:    
    for i,r in calzipcode.iterrows():
        ziplon1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Longitude']
        ziplat1 = calzipcode[calzipcode['Postal Code'] == int(z)]['Latitude']
        ziplon2 = r['Longitude']
        ziplat2 = r['Latitude']
        calzipcode.loc[i,z] = calculateDistance(ziplon1,ziplat1,ziplon2,ziplat2)

dfname = './DataPfiles/' + 'calzip' + str(int(calzipcode.iloc[startzipcodeIndex]['Postal Code'])) + "_to_" + str(int(calzipcode.iloc[endzipcodeIndex-1]['Postal Code'])) + ".p"
pickle.dump( calzipcode, open( dfname, "wb" ) )
print 'Done: ' + dfname
 
    
# geolocator = Nominatim()
# location = geolocator.geocode("94582")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)

# # >> newport_ri = (41.49008, -71.312796)
# # >>> cleveland_oh = (41.499498, -81.695391)
# print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)

In [ ]:
### This cell as well as the cell after this one are just my playground to play with different functions.  This is not part of the project. 


from pylab import plot, ylim, xlim, show, xlabel, ylabel
from numpy import linspace, loadtxt
import numpy as np

r=3.0

x = p.head()
y = pz

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

# plot(x,y)
# xlim(0,1000)

x_av = movingaverage(x, r)
# plot(x_av, y)

# xlabel("Months since Jan 1749.")
# ylabel("No. of Sun spots")
# show()
print x_av

p = df11.iloc[:,0]  #ALl Homes price
pz = df11.iloc[:,-1] 
print pz


t1 = df.iloc[:,1] 
t2 = df3.iloc[:,0]
print "mean is % 4.3F and sd is % 4.3F " % (t1.mean(),t1.std
                                            
                                            # add a new column for moving average of All Homes price


p = df11.iloc[:,0] #ALl Homes price
pz = df11.iloc[:,-1] 


window_size = 3.0 #set the number of sample to gathering centered in the middle
movingave = lambda x: np.convolve(x, np.ones(int(window_size))/float(window_size), 'same')
transformed = df11.groupby('ZipCode')
transformed['All Homes'].transform(movingave)
# df11.info()

# df11 = df11[df11['All Homes'].isnull()]
grouped = df11[['All Homes','ZipCode']].groupby('ZipCode')
b = pd.DataFrame()
newdf = pd.DataFrame()
for name,group in grouped:
    g = group.copy()
#     print group.shape
#     print "size: % 3.2F" % movingaverage(group['All Homes'],3).size
#     if sum(g.isnull()) < 0:
#         g['Moving Ave'] = movingaverage(group['All Homes'],3)

#     print g.shape
#     print g.head()

    #     if(newdf.isnull):
#         newdf = g
#     newdf = newdf.append(g)
#     b = group['All Homes']
#     a = movingaverage(group['All Homes'],3)
    
#     group["MovingAve"] = np.convolve(group['All Homes'], np.ones(int(window_size))/float(window_size), 'same')
#     newdf = newdf.append(group)
# print b
# newdf
# newdf.info()
    
    


# df11['Moving Ave'] = movingaverage(df11.iloc[:,0], 3)  #para#1 is All Homes price; para#2 is numer of sample to gather centered in the middle

# df11.head()
x                                             

In [ ]:
np.average(dfAllHomes['93063'][(-1-y):-1])

In [ ]:
import pandas as pd
#dfAllHomesDiffFromMovAve

In [ ]:
print dfAllHomesStdDevAcrossZipCode.shape
# dfAllHomesStdDevAcrossZipCode
# np.sum(dfAllHomesStdDevAcrossZipCode > 2)

In [ ]:
#dfAllHomesDiffFromMovAve.mean(axis =1 )

In [ ]:
dfAllHomesDiffFromMovAve.std(axis = 1)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

geolocator = Nominatim()
location = geolocator.geocode("94582")
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

# >> newport_ri = (41.49008, -71.312796)
# >>> cleveland_oh = (41.499498, -81.695391)
print(vincenty((geolocator.geocode("94582").latitude,geolocator.geocode("94582").longitude), (geolocator.geocode("94122").latitude,geolocator.geocode("94122").longitude)).miles)

In [ ]:
import pandas as pd
import pickle

df = pickle.load( open( ".\DataPfiles\calzip90001_to_90010.p", "rb" ) ) 

# print "abcde"
# zipcode = pd.read_csv('./DataPfiles/us_postal_codes.csv')
# calzipcode = zipcode[zipcode.State == 'California']

In [ ]:
import os
from os import path
files = [f for f in os.listdir(".\DataPfiles\\")]# if path.isfile(f)]
files
# print os.listdir(".\DataPfiles\\")

In [ ]:
PtoR = dfPriceToRent.mean()
print dfPriceToRent.mean(),dfPriceToRent.std()
print PtoR.mean(),PtoR.std()

highPtoRzip = pd.DataFrame(dfPriceToRent.mean() > 18)
highPtoRzip[highPtoRzip[0] == True]
# targetZipCodes[targetZipCodes[0] == True]

In [ ]:
# calzipcode = zipcode[zipcode.State == 'California']

# irsdata['ZipCode'] = irsdata['ZipCode'].str.strip()  #strip white space in the cell
# print sum(irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str)))  #match the rows in irsdata with calzipcode
# irsdata2007 = irsdata[irsdata['ZipCode'].isin(calzipcode['Postal Code'].apply(int).apply(str))]

# irsdata2007 = irsdata2007[irsdata2007['Range'] == ""]
# irsdata2007 = irsdata2007.drop('Range', axis=1, inplace=True)
# irsdata2007['AverageAGI'] = irsdata2007.AGI.astype(int) #/ irsdata2002['NumberOfReturns'])
# irsdata2007['NumberOfReturns'] = irsdata2007['NumberOfReturns'].astype(int)
# irsdata2007['AverageAGI'] = irsdata2007['AverageAGI'] * 1000 / irsdata2007.NumberOfReturns
# print irsdata2007['AverageAGI'].mean()


# irsdata = pd.read_csv('./DataPfiles/ZIPCode2006CA.csv',header=4, na_values=['**','--'])
# irsCol = ['ZipCode','NumberOfReturns','ExemptionsRtn','DepedentExemptionsAmt','AGI','SalariesWagesRtn','SalariesWagesAmt', \
#           'TaxableInterestRtn','TaxableInterestAmt', \
#           'TotalTaxRtn','TotalTaxAmt','ScheduleCTotalRtn','ScheduleFTotalRtn',\
#           'ScheduleARtn'
#           ]
# irsdata.columns= irsCol
# irsdata['ZipCode']

# print type(irsdata2007['Range'])
# import numpy as np
# L = [4,np.nan ,6]
# dftest = Series(L)
# dftest.apply(np.isnan)
# np.isNan(dftest)
# np.isnan()
# sum(irsdata2007.Range == "NaN")
# irsdata2007.Range 

dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
print "done"
dfPredictZipCodeFinal
# dfPredictZipCode.unstack(level=1)
# irsdatafull.unstack(level=1)

In [ ]:
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCodeFinal.unstack(level=1),irsdata2001full.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)

# b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
# # b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
# dfPredictZipCodeFinal[b]  #for testing the result

# irsdatafull = irsdata1998full
# irsdatafull = irsdatafull.append(irsdata2001full)
# irsdatafull = irsdatafull.append(irsdata2002full)
# dfPredictZipCodeFinal = pd.concat([dfPredictZipCode.unstack(level=1),irsdatafull.unstack(level=1)], axis=1)
# dfPredictZipCodeFinal = dfPredictZipCodeFinal.stack(level=1)
# dfPredictZipCodeFinal

b = dfPredictZipCodeFinal.index.get_level_values(0) == '1998-09-30'  #for testing the result
b1 = dfPredictZipCodeFinal.index.get_level_values(0) == '2001-09-30'  #for testing the result
b2 = dfPredictZipCodeFinal.index.get_level_values(0) == '2002-09-30'  #for testing the result
b4 = dfPredictZipCodeFinal.index.get_level_values(0) == '2004-09-30'  #for testing the result
b5 = dfPredictZipCodeFinal.index.get_level_values(0) == '2005-09-30'  #for testing the result
b6 = dfPredictZipCodeFinal.index.get_level_values(0) == '2006-09-30'  #for testing the result
b7 = dfPredictZipCodeFinal.index.get_level_values(0) == '2007-09-30'  #for testing the result
b8 = dfPredictZipCodeFinal.index.get_level_values(0) == '2008-09-30'  #for testing the result
b9 = dfPredictZipCodeFinal.index.get_level_values(0) == '2009-09-30'  #for testing the result
b10 = dfPredictZipCodeFinal.index.get_level_values(0) == '2010-09-30'  #for testing the result
b11 = dfPredictZipCodeFinal.index.get_level_values(0) == '2011-09-30'  #for testing the result
dfPredictZipCodeFinal[b11].head(5)  #for testing the result

In [ ]:
dfPredictZipCodeFinal.info()
# dfPredictZipCodeFinal['TotalTaxAmt']

In [ ]:
dffinaltest = dfPredictZipCodeFinal.copy()
# dffinaltest.mean
dffinaltest1 = dffinaltest.fillna(method='backfill')
dffinaltest2 = dffinaltest1.dropna(axis=0,how='any')
dffinaltest2.describe().ix[:,10:36]
# dffinaltest2['SalariesWagesAmt']

In [ ]:
dffinaltest = dfPredictZipCodeFinal.copy()
# dffinaltest.mean
dffinaltest1 = dffinaltest.dropna()
dffinaltest1.info()
dffinaltest2 = dffinaltest1.mean(axis=0)
dffinaltest2
# dffinaltest1 = dffinaltest1.ix[:,0:2]
# dffinaltest1.mean(axis=0)

In [ ]:
dffinaltest = dfPredictZipCodeFinal.copy()
a = "nan"
# nalist = dffinaltest['DepedentExemptionsAmt'].str.contains("NaN", na=True)
# dffinaltest.ix[nalist,'DepedentExemptionsAmt'] == np.nan

# dffinaltest['DepedentExemptionsAmt'] = dffinaltest['DepedentExemptionsAmt'].convert_objects(convert_numeric=True)
dffinaltest = dffinaltest.convert_objects(convert_numeric=True)
# dffinaltest['DependentExemptionsAmt'] = dffinaltest['DependentExemptionsAmt'].astype(float)
dffinaltest2 = dffinaltest.fillna(dffinaltest.mean(numeric_only=True))

dffinaltest2['DepedentExemptionsAmt']

# dffinaltest['DepedentExemptionsAmt']
# pd.isnull(dffinaltest['DepedentExemptionsAmt'])

# dffinaltest['DepedentExemptionsAmt'].fillna("abc")
# dffinaltest['DepedentExemptionsAmt']
# a = np.nan
# np.isnan(a)
# type(a)